In [12]:
!pip install yfinance
#https://github.com/ranaroussi/yfinance
#https://finance.yahoo.com/

In [13]:
import yfinance as yf
import numpy as np
import sys
import pandas as pd
import requests 
from pandas_datareader import data as pdr
pd.options.display.max_columns = None

In [14]:
def mergedataframes(df1 , df2, stringdf1 = None, stringdf2 = None):
  dfmerged = pd.merge(df1, df2, on="Date",suffixes = (stringdf1,stringdf2))
  return dfmerged

In [15]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = "http://www.fundamentus.com.br/detalhes.php?papel="

html = requests.get(url,headers=headers)
tickers = pd.read_html(html.content)[0]

tickers

,Papel,Nome Comercial,Razão Social
0,AALR3,ALLIAR,CENTRO DE IMAGEM DIAGNOSTICOS S.A.
1,ABCB3,ABC Brasil,BANCO ABC BRASIL S/A
2,ABCB4,ABC Brasil,BANCO ABC BRASIL S/A
3,ABEV3,AMBEV S/A,AMBEV S.A.
4,ABRE3,SOMOS EDUCA,SOMOS EDUCAÇÃO S.A.
...,...,...,...
1017,WLMM3,WLM,WLM INDÚSTRIA E COMÉRCIO S/A
1018,WLMM4,WLM,WLM INDÚSTRIA E COMÉRCIO S/A
1019,WMBY3,WEMBLEY SOCIEDADE ANÔNIMA,WEMBLEY SOCIEDADE ANÔNIMA
1020,WSON33,Wilson Sons,WILSON SONS LIMITED


#Single Processing

Aproximadamente, 12 minutos para extrair o histórico de todas as ações listadas na B3

In [ ]:
Choose_Stocks = tickers["Papel"].values + ".SA"

df_final = pd.DataFrame()

for i in Choose_Stocks:
  Stock = yf.Ticker(i)
  yf.pdr_override()
  dfStock = Stock.history(period="max")
  dfStock["Papel"] = i
  df_final = df_final.append(dfStock)

df_final = df_final[df_final.columns[:-1]]
df_final

#MultiProcessing

Aproximadamente, 55 segundos para extrair o histórico de todas as ações listadas na B3

In [17]:
def scrape_Yf_Finance(Choose_Stocks):

  Stock = yf.Ticker(Choose_Stocks)
  yf.pdr_override()
  dfStock = Stock.history(period="max")
  dfStock["Papel"] = Choose_Stocks
    
  return dfStock

In [ ]:
from multiprocessing import Pool

Choose_Stocks = tickers["Papel"].values + ".SA"
p = Pool(50)
df_final = p.map(scrape_Yf_Finance, Choose_Stocks)
p.terminate()
p.join()

df_final = pd.concat(df_final)
df_final = df_final[df_final.columns[:-1]]

In [20]:
df_final.to_csv("base.csv", sep = ";", decimal=',')